In [1]:
# Import library
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
import numpy as np
import psycopg2

### Loading data

In [2]:
def loading_data_from_db(database_name):
    # Database connection parameters
    host = "34.129.179.229"
    database = "project"
    user = "postgres"
    password = "Iti<FVBpR:05tTOU"
    port = "5432"

    # SQL query to fetch data
    query = f"SELECT * FROM {database_name}"

    # Connect to PostgreSQL and fetch data
    try:
        # Establish the connection
        conn = psycopg2.connect(
            host=host,
            database=database,
            user=user,
            password=password,
            port=port
        )
        # Load data into a DataFrame
        df = pd.read_sql_query(query, conn)
        
        # Close the connection
        conn.close()

        print(f"{database_name} downloaded successfully!")
        return df

    except Exception as e:
        print(f"An error occurred: {e}")

In [3]:
heat_data = loading_data_from_db("heat_data")
global_climate = loading_data_from_db("global_climate_data")
forest_trend = loading_data_from_db("forest_trend")
world_temp_data = loading_data_from_db("world_temp_data")

/var/folders/8v/_f0s3ppn27g2ss5m_lvb311m0000gn/T/ipykernel_39021/1946496499.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


heat_data downloaded successfully!
global_climate_data downloaded successfully!
forest_trend downloaded successfully!
world_temp_data downloaded successfully!


-----

### Global Climate

In [4]:
global_climate.head()

,year,avg_temp,lower_bound,upper_bound,co2,methane,ghg_emission
0,1850,-0.055414,-0.226959,0.116131,196847600.0,1.239410e+09,4.219061e+09
1,1851,0.128948,-0.049570,0.307466,198778540.0,1.244053e+09,4.409311e+09
2,1852,0.132899,-0.047085,0.312882,207595090.0,1.250690e+09,4.465078e+09
3,1853,0.091943,-0.067712,0.251598,217236980.0,1.258188e+09,4.532813e+09
4,1854,0.070777,-0.070414,0.211968,255038560.0,1.270684e+09,4.594584e+09


In [5]:
world_temp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         176 non-null    int64  
 1   avg_temp     176 non-null    float64
 2   lower_bound  176 non-null    float64
 3   upper_bound  176 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 5.6 KB


In [6]:
fig = go.Figure(layout=go.Layout(template='plotly_white'))

# Add traces for each data series

fig.add_trace(go.Scatter(
    x=world_temp_data['year'],
    y=world_temp_data['upper_bound'],
    mode='lines',
    name='Upper Bound',
    line=dict(width=0),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=world_temp_data['year'],
    y=world_temp_data['lower_bound'],
    mode='lines',
    line=dict(width=0),
    fillcolor = 'rgba(68, 68, 68, 0.3)',
    fill = 'tonexty',
    name='Lower Bound',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=world_temp_data['year'],
    y=world_temp_data['avg_temp'],
    mode='lines',
    name='World Average Temperature',
    line=dict(color='red')
))

fig.update_layout(
    title='Average Temperature with Confidence Interval',
    xaxis_title='Year',
    yaxis_title='Temperature (Celcius)',
    hovermode='x unified', # Shows hover info for all traces at a given x-value
    legend_title_text='Legend'
)

Scientists worldwide have tracked Earth's temperature for decades. The data is clear: our planet's average temperature is rising at an unprecedented rate, particularly since the mid-20th century. This isn't just a slight fluctuation; it's a significant shift in our global climate system.

Line chart tracks the global average temperature anomaly (the deviation from a baseline average, shown in degrees Celsius) from the year 1850 up to the early 21st century. The red line represents the estimated world average temperature for each year, while the shaded area around the line indicates the 95% confidence interval for that year.

The chart clearly shows year-to-year fluctuations, but the dominant feature is a distinct upward trend, particularly after roughly 1950. This indicates a significant increase in the world's average temperature over time, which is a key indicator of global warming. The confidence interval narrows over time, reflecting improved accuracy in temperature measurements as technology advances. The chart effectively communicates the long-term trend of rising temperatures.

This warming is primarily driven by the increasing concentration of greenhouse gases (GHGs) in our atmosphere, released by human activities. Think of these gases like a thickening blanket around the Earth, trapping more heat. Carbon Dioxide (CO2) from burning fossil fuels (coal, oil, gas) is the main contributor, but Methane (CH4) from agriculture and gas leaks, and Nitrous Oxide (N2O) are also powerful heat-trappers.    

In [7]:

# Create the figure
fig = go.Figure(layout=go.Layout(template='plotly_white'))

# Add traces for each data series
fig.add_trace(go.Scatter(
    x=global_climate['year'], 
    y=global_climate['avg_temp'], 
    mode='lines', 
    name='Avg Temperature (°C)', 
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=global_climate['year'], 
    y=global_climate['co2'] / 1e10, 
    mode='lines', 
    name='CO₂ Emissions (x10⁻¹⁰)', 
    line=dict(color='blue')
))

fig.add_trace(go.Scatter(
    x=global_climate['year'], 
    y=global_climate['methane'] / 1e10, 
    mode='lines', 
    name='Methane Emissions (x10⁻¹⁰)', 
    line=dict(color='orange')
))

fig.add_trace(go.Scatter(
    x=global_climate['year'], 
    y=global_climate['ghg_emission'] / 1e10, 
    mode='lines', 
    name='GHG Emissions (x10⁻¹⁰)', 
    line=dict(color='green')
))

# Customize layout
fig.update_layout(
    title='Temperature and Emissions Over Time',
    xaxis_title='Year',
    yaxis_title='Values (Normalized)',
    hovermode='x unified',
    legend_title_text='Legend',
)

# Show the plot
fig.show()

Chart plots four different variables over time, from 1850 to the early 21st century:

- Average Global Temperature (Red Line)
- CO₂ Emissions (Blue Line)
- Methane Emissions (Orange Line)
- Total Greenhouse Gas (GHG) Emissions (Green Line)

The vertical axis shows normalized values, meaning the data for each line has been scaled to fit on a common axis to compare their trends rather than their absolute amounts (though the legend notes the original scale for emissions).

The key takeaway is that all four metrics show a clear increasing trend over the period, particularly accelerating from the mid-20th century (around 1950) onwards. The chart visually suggests a strong correlation between the rise in greenhouse gas emissions (CO₂, Methane, and total GHGs) and the increase in average global temperature.

---

Carbon Emission by sector

In [8]:
carbon_emission_by_sector = pd.read_csv("https://ourworldindata.org/grapher/co-emissions-by-sector.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
carbon_emission_by_sector.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6560 entries, 0 to 6559
Data columns (total 12 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Entity                                        6560 non-null   object 
 1   Code                                          6208 non-null   object 
 2   Year                                          6560 non-null   int64  
 3   buildings_co2_emissions                       6421 non-null   float64
 4   industry_co2_emissions                        6305 non-null   float64
 5   land_use_change_and_forestry_co2_emissions    6557 non-null   float64
 6   other_fuel_combustion_co2_emissions           6421 non-null   float64
 7   transport_co2_emissions                       6421 non-null   float64
 8   manufacturing_and_construction_co2_emissions  6421 non-null   float64
 9   fugitive_co2_emissions                        2270 non-null   f

In [9]:
aus_co2_by_sector = carbon_emission_by_sector[carbon_emission_by_sector['Entity'] == 'Australia']
aus_co2 = aus_co2_by_sector[aus_co2_by_sector['Year'] == 2021]

In [10]:
# Sum all sectors into total
aus_co2['Total'] = aus_co2.iloc[:, 3:].sum(axis=1)

# Calculate percentage of each sector
sector_percentages = aus_co2.iloc[:, 3:-1].div(aus_co2['Total'], axis=0) * 100


/var/folders/8v/_f0s3ppn27g2ss5m_lvb311m0000gn/T/ipykernel_39021/2344615319.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
sector_percentages.to_csv('sector_percentages.csv', index=False, encoding='utf-8')

In [12]:
sector_percentages.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 351 to 351
Data columns (total 9 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   buildings_co2_emissions                       1 non-null      float64
 1   industry_co2_emissions                        1 non-null      float64
 2   land_use_change_and_forestry_co2_emissions    1 non-null      float64
 3   other_fuel_combustion_co2_emissions           1 non-null      float64
 4   transport_co2_emissions                       1 non-null      float64
 5   manufacturing_and_construction_co2_emissions  1 non-null      float64
 6   fugitive_co2_emissions                        1 non-null      float64
 7   electricity_and_heat_co2_emissions            1 non-null      float64
 8   aviation_and_shipping_co2_emissions           1 non-null      float64
dtypes: float64(9)
memory usage: 80.0 bytes


In [21]:
sector_data = sector_percentages

labels = [
    'Buildings',
    'Industry',
    'Land Use Change & Forestry',
    'Other Fuel Combustion',
    'Transport',
    'Manufacturing & Construction',
    'Fugitive Emissions',
    'Electricity & Heat',
    'Aviation & Shipping'
]
values = sector_data.iloc[0].tolist()

# Use ⁠ hole ⁠ to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])

# Add title
fig.update_layout(title_text="CO2 Emissions by Sector in Australia (2021)")

fig.show()

The donut chart illustastrates the percentage of CO2 emissions by sector in Australia in 2021. The largest contribution is comes from the electricity and heat secotr, followed by transportation.

-----

### Forest Trend

In [14]:
forest_trend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country name            248 non-null    object 
 1   country code            248 non-null    object 
 2   percentage_change       248 non-null    float64
 3   percentage_change_norm  248 non-null    float64
dtypes: float64(2), object(2)
memory usage: 7.9+ KB


In [15]:
fig = go.Figure(data=go.Choropleth(
    locations=forest_trend['country code'],  # ISO-3 country codes
    z=forest_trend['percentage_change'],  # Percentage change in forest area
    text=forest_trend['country name'],  # Hover text
    colorscale="RdYlGn",  # Color scale
    colorbar_title="Percentage Change"
))

fig.update_layout(
    title_text="Percentage Change in Forest Area (2000-2022)",
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    )
)

fig.show()

In [16]:
kde = gaussian_kde(forest_trend['percentage_change_norm'])
x_min = forest_trend['percentage_change_norm'].min()
x_max = forest_trend['percentage_change_norm'].max()
x_range = np.linspace(x_min - (x_max-x_min)*0.1, x_max + (x_max-x_min)*0.1, 500) # Evaluate at 500 points
y = kde(x_range)

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=forest_trend['percentage_change_norm'],
    nbinsx=30,
    marker_color='lightblue',
    histnorm='probability density',
    name='Frequency (Histogram)'
))

fig.add_trace(go.Scatter(
    x=x_range,
    y=y,
    mode='lines',
    name='KDE',
    line=dict(color='blue')
))

fig.update_layout(
    title='Distribution of Forest Trend (2000-2020)',
    title_font_size=16,
    xaxis_title='Trend',
    yaxis_title='Frequency',
    xaxis=dict(title_font=dict(size=14)),
    yaxis=dict(title_font=dict(size=14)),
    bargap=0.05,  # Optional: space between bars
    template='plotly_white'  # Set theme to minimal
)

fig.show()

The visualizations together offer a compelling overview of global forest area trends from 2000 to 2022, with meaningful implications for understanding and responding to climate change. The first visualization, a global heatmap, illustrates the percentage change in forest area by country, where red tones signal serious deforestation, yellow reflects minimal change, and green represents positive forest growth. This spatial distribution reveals that countries in Central Africa, Southeast Asia, and South America have experienced significant forest loss, while some parts of Europe and Asia, such as China, have seen modest forest gains due to reforestation initiatives. The second graph, a distribution plot of normalized forest trends, shows that most countries fall within moderate ranges of forest change, with a majority tending toward slight gains or losses and fewer countries at the extremes. The peak around 0.4 suggests that moderate changes in forest cover are most common globally.

These forest changes are closely linked to the broader context of climate change. Forests act as major carbon sinks, absorbing billions of tons of CO₂ from the atmosphere each year. Therefore, the loss of forest cover—particularly in tropical regions—directly reduces the Earth’s ability to sequester carbon, exacerbating global warming. In regions where large-scale deforestation is taking place, such as the Amazon or Central Africa, not only is stored carbon released back into the atmosphere, but local climates are also affected through changes in rainfall patterns, temperature variability, and biodiversity loss. On the flip side, forest gains in some countries can help mitigate climate change by enhancing carbon capture, promoting ecological stability, and improving resilience against extreme weather events. However, the uneven global trend highlighted by the distribution suggests that while some nations are fighting climate degradation through forest restoration, widespread deforestation elsewhere continues to undermine global climate goals.

Together, these visualizations emphasize the essential role of forest conservation and restoration in addressing climate change. They reinforce the urgent need for coordinated international policies and locally adapted strategies that can reduce forest loss, enhance reforestation, and promote sustainable land use, all of which are vital for limiting global warming and securing environmental sustainability.

----

https://www.abc.net.au/news/2025-02-23/extreme-heat-greater-sydney-cities-australia/104961478

A new study suggests the prevalence of extreme heat in Australia's cities is being under-reported.

Temperatures above 35 degrees Celsius were far more common across parts of Greater Sydney than initially reported by the Bureau of Meteorology, according to the researchers.

The findings may present implications for cities across Australia, with climate modelling predicting extreme heat is on track to occur more frequently.

### Heat Data

In [17]:
heat_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53717 entries, 0 to 53716
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   max_temp  53713 non-null  float64
 1   date      53717 non-null  object 
 2   city      53717 non-null  object 
 3   is_hot    53717 non-null  bool   
dtypes: bool(1), float64(1), object(2)
memory usage: 1.3+ MB


In [18]:
heat_data['date'] = pd.to_datetime(heat_data['date'])
heat_data['year'] = heat_data['date'].dt.year

# Count hot day
hot_days = heat_data[heat_data['is_hot']]
hot_days_per_year = hot_days.groupby(['city', 'year']).size().reset_index(name='hot_day_count')

In [20]:
import plotly.graph_objects as go

# Transform data into a format suitable for heatmap: city as rows, year as columns
pivot = hot_days_per_year[(hot_days_per_year['year'] >= 2014) & (hot_days_per_year['year'] <= 2024)].pivot(index='city', columns='year', values='hot_day_count')

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=pivot.values,
    x=pivot.columns,
    y=pivot.index,
    colorscale='YlOrRd',
    colorbar=dict(title="Hot Days"),
    zmin=0,  # Optional: Set minimum value for color scale
    text=pivot.values,  # Add text for each cell
    texttemplate="%{text:.0f}"  # Format text as integers
))

# Update layout
fig.update_layout(
    title="Number of Hot Days per Year by City",
    xaxis_title="Year",
    yaxis_title="City",
    template="plotly_white"
)

# Show the heatmap
fig.show()


The heatmap illustrates the number of hot days (defined as days with temperatures exceeding 30°C) across four major Australian cities—Sydney, Melbourne, Brisbane, and Adelaide—over the years 2014 to 2024. The data highlights a key effect of climate change: an observable increase in the frequency and intensity of hot days in many areas. Brisbane shows the most consistent and significant rise, reaching a peak of 121 hot days in 2019, indicating that northern cities are particularly vulnerable to extreme temperature increases. Adelaide also demonstrates an upward trend since 2017, with a notable rise to 72 hot days in 2018 and a return to 63 hot days by 2024, suggesting a warming shift in its climate. Sydney and Melbourne, traditionally more temperate cities, exhibit more yearly fluctuations but still reflect rising heat patterns, with both cities showing increases in hot days in recent years after some dips.

These variations suggest that interannual weather phenomena, such as El Niño or La Niña, may influence yearly temperatures, but the overall trend aligns with the growing impact of global warming. The data indicates not only that hot days are becoming more frequent but also that heatwaves and prolonged periods of extreme temperatures are becoming more common. This rise in heat can have serious consequences for human health, energy demand, infrastructure durability, and natural ecosystems. Overall, the increase in the number of hot days is a clear indicator that climate change is intensifying, with noticeable regional differences, calling for urgent adaptation and mitigation strategies at both local and national levels.

----

Other impact

Health Risks

Beyond heatstroke, climate change impacts our health through worse air quality (bushfire smoke, pollution), potentially changing patterns of infectious diseases carried by insects, and impacts on mental health from experiencing climate-related events.

Nature Under Pressure

The natural environments we love are suffering. Increased bushfire risk threatens forests like the Blue Mountains and the Dandenongs. Unique ecosystems like the Alpine regions face reduced snow cover. Iconic wildlife, from koalas struggling in heatwaves to disruptions in marine life, are impacted. This loss affects biodiversity and the natural beauty we value.

Economic Ripples

Climate impacts have economic costs: rising insurance premiums, expensive repairs to infrastructure after extreme weather, impacts on agriculture affecting food prices, and potential effects on tourism.

-----

### Nature: Our Vulnerable Shield, Our Essential Ally

The nature we aim to conserve is not just a victim of climate change; it's also a crucial part of the solution. Healthy, functioning ecosystems provide essential services that help us both mitigate climate change and adapt to its impacts.

Carbon Sinks: Forests, wetlands, mangroves, and healthy soils absorb and store vast amounts of CO2, helping to regulate the climate. Protecting and restoring these is vital.
Natural Cooling: Trees and green spaces in our cities provide shade and cool the air through evapotranspiration, reducing the urban heat island effect and lowering energy demand for cooling.
Flood & Storm Protection: Wetlands, mangroves, and coastal reefs act as natural buffers, absorbing storm surges and heavy rainfall, protecting communities and infrastructure.
Biodiversity & Resilience: Diverse ecosystems are generally more resilient to change. Protecting biodiversity helps maintain the health and adaptability of these natural systems.
Conserving nature isn't separate from tackling climate change – it's integral. Protecting the green spaces within and around Melbourne and Sydney (from urban parks and river corridors like the Yarra or Parramatta Rivers, to surrounding bushland and coastal zones) directly contributes to our climate resilience and well-being.

### Taking Action Together
The challenge is significant, but despair is not an option. We have the knowledge and increasingly the tools to make a difference. As urban residents, our collective actions and choices matter.

How Can We Help Conserve Nature in a Changing Climate?
- Support Local Conservation: Volunteer with or donate to groups working on habitat restoration, tree planting, or wildlife protection in your area (e.g., Bushcare, Landcare, park friends groups).
- Make Nature-Friendly Choices at Home: Create habitat in your own space (even a balcony!), reduce water use, minimise waste, choose native plants.
- Live Sustainably: Reduce your carbon footprint through energy efficiency, sustainable transport (walking, cycling, public transport), conscious consumption, and reducing meat intake.
- Advocate for Change: Support policies that promote renewable energy, protect natural areas, and invest in climate adaptation. Talk to friends, family, and elected officials.
- Value Green Spaces: Actively use and advocate for the protection and enhancement of parks, urban forests, and waterways in Melbourne and Sydney.
- Using Ecoshpere application: Ecoshpere is a free app that provides tips to help you live more sustainably and support local conservation efforts by planting trees and restoring habitats.

Every action that supports nature builds resilience. Protecting and restoring ecosystems is one of the most effective ways we can invest in a safer, healthier, and more stable future for ourselves and for the unique natural heritage of Australia.

Let's work together to ensure both our vibrant cities and the nature that sustains them can thrive, even in the face of climate change.